Check azure-ai-ml


In [3]:
pip show azure-ai-ml

Name: azure-ai-ml
Version: 1.29.0
Summary: Microsoft Azure Machine Learning Client Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python
Author: Microsoft Corporation
Author-email: azuresdkengsysadmins@microsoft.com
License: MIT License
Location: c:\Users\Tony\AppData\Local\Programs\Python\Python313\Lib\site-packages
Requires: azure-common, azure-core, azure-mgmt-core, azure-monitor-opentelemetry, azure-storage-blob, azure-storage-file-datalake, azure-storage-file-share, colorama, isodate, jsonschema, marshmallow, pydash, pyjwt, pyyaml, strictyaml, tqdm, typing-extensions
Required-by: 
Note: you may need to restart the kernel to use updated packages.


Connect to my workspace

In [4]:
#import required libraries
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
#Enter    details    of    your    Azure    Machine    Learning    workspace
subscription_id = 'c6cdfd9c-3767-4be7-a343-0acd27ddefb9'
resource_group = 'rg-tshen-0917'
workspace = 'ml-ws-tshen-0917'
#connect to the workspace
ml_client    =    MLClient(DefaultAzureCredential(),    subscription_id,
resource_group,    workspace)
ml_client

MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x000002A39BE19FD0>,
         subscription_id=c6cdfd9c-3767-4be7-a343-0acd27ddefb9,
         resource_group_name=rg-tshen-0917,
         workspace_name=ml-ws-tshen-0917)

Write train.py

In [7]:
%%writefile ./src/train.py
# train.py
# import necessary libraries
import pandas as pd
import argparse
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# get parameters
parser = argparse.ArgumentParser("train")
parser.add_argument("--training_data", type=str, help="Path to training data")
parser.add_argument("--n_estimators", type=int, default=100, help="Number of trees in the forest")
parser.add_argument("--model_output", type=str, help="Path of output model")
parser.add_argument("--test_size", type=float, default=0.30, help="test size")
parser.add_argument("--random_state", type=int, default=None, help="random state")

args = parser.parse_args()
print(args)

training_data = args.training_data
model_output = args.model_output
n_estimators = args.n_estimators
test_size = args.test_size
random_state = args.random_state

# Load your dataset
df = pd.read_csv(training_data)  # Replace with your actual file

# Assume the last column is the target
# X = data.iloc[:, :-1]  # Features
# y = data.iloc[:, -1]   # Target

# Separate features and labels
X, y = (
    df[
        [
            "Pregnancies",
            "PlasmaGlucose",
            "DiastolicBloodPressure",
            "TricepsThickness",
            "SerumInsulin",
            "BMI",
            "DiabetesPedigree",
            "Age",
        ]
    ].values,
    df["Diabetic"].values,
)

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=test_size, random_state=random_state
)

# Initialize and train the classifier
model = RandomForestClassifier(n_estimators=n_estimators, random_state=random_state)
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred))

Overwriting ./src/train.py


Run train.py

In [8]:
!python ./src/train.py --training_data ./diabetes-data/diabetes.csv --model_output ./model/random-forest.save

Namespace(training_data='./diabetes-data/diabetes.csv', n_estimators=100, model_output='./model/random-forest.save', test_size=0.3, random_state=None)
Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.95      0.94      1990
           1       0.89      0.89      0.89      1010

    accuracy                           0.93      3000
   macro avg       0.92      0.92      0.92      3000
weighted avg       0.93      0.93      0.93      3000



Write pytorch_iris.py

In [23]:
%%writefile ./src/pytorch_iris.py
# pytorch_iris.py
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import argparse
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# get parameters
parser = argparse.ArgumentParser("train")
parser.add_argument("--training_data", type=str, default='./iris-data/iris.csv', help="Path to training data")
parser.add_argument("--reg_rate", type=float, default=0.01)
parser.add_argument("--model_output", type=str, default='./model/iris.save', help="Path of output model")
parser.add_argument("--test_size", type=float, default=0.30, help="test size")
parser.add_argument("--random_state", type=int, default=0, help="random state")
parser.add_argument("--n_epoch", type=int, default=50, help="number of epochs")
parser.add_argument("--learning_rate", type=float, default=0.01, help="learning rate")
args = parser.parse_args()
print(args)

training_data = args.training_data
reg_rate = args.reg_rate
model_output = args.model_output
test_size = args.test_size
random_state = args.random_state
n_epoch = args.n_epoch
learning_rate = args.learning_rate

# Load Iris dataset
df = pd.read_csv(training_data)  # Replace with your actual file
X = df.drop("species", axis=1).values
y = LabelEncoder().fit_transform(df["species"])
# Assume the last column is the target
# X = df.iloc[:, :-1]  # Features
# y = df.iloc[:, -1]   # Target

# Preprocess
scaler = StandardScaler()
X = scaler.fit_transform(X)
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

# Define model
class IrisClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(4, 16),
            nn.ReLU(),
            nn.Linear(16, 3)
        )

    def forward(self, x):
        return self.net(x)

model = IrisClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=reg_rate)

# Training loop
for epoch in range(50):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

# Save model
# torch.save(model.state_dict(), "iris_model.pt")
torch.save(model.state_dict(), model_output)

# Evaluate
with torch.no_grad():
    preds = torch.argmax(model(X_test), dim=1)
    acc = (preds == y_test).float().mean()
    print(f"Test Accuracy: {acc:.4f}")

Overwriting ./src/pytorch_iris.py


Run pytorch_iris.py

In [24]:
!python ./src/pytorch_iris.py --training_data ./iris-data/iris.csv

Namespace(training_data='./iris-data/iris.csv', reg_rate=0.01, model_output='./model/iris.save', test_size=0.3, random_state=0, n_epoch=50, learning_rate=0.01)
Epoch 1, Loss: 1.1613
Epoch 2, Loss: 1.1084
Epoch 3, Loss: 1.0578
Epoch 4, Loss: 1.0091
Epoch 5, Loss: 0.9624
Epoch 6, Loss: 0.9177
Epoch 7, Loss: 0.8750
Epoch 8, Loss: 0.8342
Epoch 9, Loss: 0.7952
Epoch 10, Loss: 0.7584
Epoch 11, Loss: 0.7237
Epoch 12, Loss: 0.6909
Epoch 13, Loss: 0.6599
Epoch 14, Loss: 0.6309
Epoch 15, Loss: 0.6037
Epoch 16, Loss: 0.5780
Epoch 17, Loss: 0.5540
Epoch 18, Loss: 0.5313
Epoch 19, Loss: 0.5101
Epoch 20, Loss: 0.4905
Epoch 21, Loss: 0.4720
Epoch 22, Loss: 0.4547
Epoch 23, Loss: 0.4387
Epoch 24, Loss: 0.4239
Epoch 25, Loss: 0.4103
Epoch 26, Loss: 0.3975
Epoch 27, Loss: 0.3855
Epoch 28, Loss: 0.3743
Epoch 29, Loss: 0.3636
Epoch 30, Loss: 0.3536
Epoch 31, Loss: 0.3440
Epoch 32, Loss: 0.3347
Epoch 33, Loss: 0.3259
Epoch 34, Loss: 0.3177
Epoch 35, Loss: 0.3099
Epoch 36, Loss: 0.3025
Epoch 37, Loss: 0.295

Write tensorflow_iris.py

In [2]:
%%writefile ./src/tensorflow_iris.py
# tensorflow_iris.py
import tensorflow as tf
import pandas as pd
import argparse
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# get parameters
parser = argparse.ArgumentParser("train")
parser.add_argument("--training_data", type=str, default='./iris-data/iris.csv', help="Path to training data")
parser.add_argument("--reg_rate", type=float, default=0.01)
parser.add_argument("--model_output", type=str, default='./model/iris_tf.keras', help="Path of output model, file extension .keras instead of .h5 required")
parser.add_argument("--test_size", type=float, default=0.30, help="test size")
parser.add_argument("--random_state", type=int, default=0, help="random state")
parser.add_argument("--n_epoch", type=int, default=50, help="number of epochs")
parser.add_argument("--learning_rate", type=float, default=0.01, help="learning rate")
parser.add_argument("--batch_size", type=int, default=16, help="batch size")
parser.add_argument("--gpu", type=str, default="no", help="if gpu available yes/no")
args = parser.parse_args()
print(args)

training_data = args.training_data
reg_rate = args.reg_rate
model_output = args.model_output
test_size = args.test_size
random_state = args.random_state
n_epoch = args.n_epoch
learning_rate = args.learning_rate
batch_size = args.batch_size
gpu = args.gpu
print(f"GPU enabled: {gpu}")

# Load Iris dataset
df = pd.read_csv(training_data)  # Replace with your actual file
X = df.drop("species", axis=1).values
y = LabelEncoder().fit_transform(df["species"])

# Preprocess
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

# Build model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(batch_size, activation='relu', input_shape=(4,)),
    tf.keras.layers.Dense(3, activation='softmax')
])

model.compile(optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy'])

# Train with GPU if available
if gpu.lower() == "yes":    
    print("Training with GPU...")
    with tf.device('/GPU:0'):
        model.fit(X_train, y_train, epochs=n_epoch, batch_size=batch_size, verbose=1)
else:
    print("Training with CPU...")   
    model.fit(X_train, y_train, epochs=n_epoch, batch_size=batch_size, verbose=1)

# Save model
model.save(model_output) # Evaluate
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc:.4f}")

Overwriting ./src/tensorflow_iris.py


Run tensorflow_iris.py

In [9]:
!python ./src/tensorflow_iris.py --training_data ./iris-data/iris.csv --gpu yes --n_epoch 80 --learning_rate 0.01 --batch_size 16

2025-09-20 18:09:15.107229: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Namespace(training_data='./iris-data/iris.csv', reg_rate=0.01, model_output='./model/iris_tf.keras', test_size=0.3, random_state=0, n_epoch=80, learning_rate=0.01, batch_size=16, gpu='yes')
GPU enabled: yes
/home/tshen/dp-100/mslearn-azure-ml/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1758409759.944554   47390 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 w

Verify GPU access

In [2]:
import tensorflow as tf
print(tf.config.list_physical_devices())

2025-09-19 19:30:31.753348: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
